### The Idea : Analyse Covid-19 Data by Age groups. 
#Step 0: Import Libraries and load data from data.gov.ie
   
#Step 1: Select only necessary columns and rows

#Step 2: Clean data before transform to tidy data format

#Step 3: Create two new columns storing Daily Confirmed Cases and Daily Hospitalised Cases

#Step 4: Split the final dataset into two datasets, one for Hospitalised Cases and one for Confirmed Cases. Then, transform these datasets into tidy formats.

#Step 5: Merge two tidy datasets

#Step 6: The same idea as Step 3, but here, we need to create daily data column for each day for each age group because there is only cumulative data by date

#Step 7: Merge all above datasets again

#Step 8: Handling NaN values

#Step 9: Export to csv file

#### Step 0. Import libraries and load data from data.gov.ie

In [1]:
import pandas as pd
import datetime as dt

URL = "https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"

df = pd.read_csv(URL)

df.StatisticsProfileDate = pd.to_datetime(df.StatisticsProfileDate)

df = df.sort_values(by="StatisticsProfileDate")


#### Based on updated information on https://data.gov.ie/dataset/covid-19-hpsc-detailed-statistics-profile:

- The primary Date applies to:ConfirmedCovidCases, TotalConfirmedCovidCases, ConfirmedCovidDeaths, TotalCovidDeaths, ConfirmedCovidRecovered, TotalCovidRecovered. The StatisticProfileDate applies to the remaining fields. So, we will only filter out some necessary columns for Data Analysis.

- Due to some technical issues over the Christmas period, updates of all detailed profile will be paused. So, we will only select data before 22 December, 2021 for Data Analysis

#### Step 1: Select only necessary columns and rows

In [2]:
df_filtered = df.loc[:,['StatisticsProfileDate','CovidCasesConfirmed', 'HospitalisedCovidCases','Median_Age', 
                        'HospitalisedAged5', 'HospitalisedAged5to14','HospitalisedAged15to24', 
                        'HospitalisedAged25to34','HospitalisedAged35to44', 'HospitalisedAged45to54',
                        'HospitalisedAged55to64','HospitalisedAged65to74','HospitalisedAged85up','HospitalisedAged75to84',
                        'Aged1to4','Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44', 
                        'Aged45to54','Aged55to64','Aged65to74','Aged75to84', 'Aged85up']]

end_date = '2021-12-22'
df_filtered = df_filtered[df_filtered.StatisticsProfileDate < end_date]

start_date = '2020-03-02'
df_filtered = df_filtered[df_filtered.StatisticsProfileDate >= start_date]

#### Step 2. Clean data before transform to tidy data format

In [3]:
df_final = df_filtered.copy().reset_index(drop=True)
df_final = df_final.rename({'HospitalisedAged5':'HospitalisedAged1to4'}, axis = 'columns')

#### Step 3. Create two new columns storing Daily Confirmed Cases and Daily Hospitalised Cases

In [4]:
df_final["CovidCasesConfirmedByDay"] = df_final["CovidCasesConfirmed"].diff()
df_final.loc[df_final["StatisticsProfileDate"]== start_date,"CovidCasesConfirmedByDay"] = df_final.loc[df_final["StatisticsProfileDate"]== start_date,"CovidCasesConfirmed"]

df_final["HospitalisedCovidCasesByDay"] = df_final["HospitalisedCovidCases"].diff()
df_final.loc[df_final["StatisticsProfileDate"]== start_date,"HospitalisedCovidCasesByDay"] = df_final.loc[df_final["StatisticsProfileDate"]== start_date,"HospitalisedCovidCases"]

#### Step 4. Split the final dataset into two datasets, one for Hospitalised Cases and one for Confirmed Cases. Then, transform these datasets into tidy formats.

In [5]:
df_hospital = df_final.loc[:,['StatisticsProfileDate','CovidCasesConfirmed', 'CovidCasesConfirmedByDay',
                              'HospitalisedCovidCases', 'HospitalisedCovidCasesByDay','Median_Age', 
                              'HospitalisedAged1to4', 'HospitalisedAged5to14',
                              'HospitalisedAged15to24', 'HospitalisedAged25to34',
                              'HospitalisedAged35to44', 'HospitalisedAged45to54',
                              'HospitalisedAged55to64', 'HospitalisedAged65to74',
                              'HospitalisedAged75to84', 'HospitalisedAged85up' ]]

df_hospital.columns = df_hospital.columns.str.replace("HospitalisedAged","")

id_vars = ['StatisticsProfileDate','CovidCasesConfirmed',  'CovidCasesConfirmedByDay', 'HospitalisedCovidCases', 'HospitalisedCovidCasesByDay','Median_Age']

df_hos_melt = pd.melt(df_hospital, id_vars=id_vars,value_name="HospitalisedCases",var_name="Age")

In [6]:
df_case = df_final.loc[:,['StatisticsProfileDate','CovidCasesConfirmed', 'CovidCasesConfirmedByDay', 'HospitalisedCovidCases',  'HospitalisedCovidCasesByDay','Median_Age',
                          'Aged1to4','Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44', 
                          'Aged45to54','Aged55to64',  'Aged65to74','Aged75to84', 'Aged85up' ]]

df_case.columns = df_hospital.columns.str.replace("Aged","")

id_vars = ['StatisticsProfileDate','CovidCasesConfirmed',  'CovidCasesConfirmedByDay', 'HospitalisedCovidCases', 'HospitalisedCovidCasesByDay','Median_Age']

df_case_melt = pd.melt(df_case, id_vars=id_vars,value_name="Cases",var_name="Age")

#### Step 5. Merge two tidy datasets

In [7]:
df_melt = pd.merge(df_case_melt, df_hos_melt,how="inner",on=['StatisticsProfileDate',
       'CovidCasesConfirmed', 'CovidCasesConfirmedByDay', 'HospitalisedCovidCases', 'HospitalisedCovidCasesByDay','Median_Age','Age'])

#### Step 6. The same idea as Step 3, but here, we need to create daily data column for each day for each age group because there is only cumulative data by date, 

In [8]:
df_melt_1to4 = df_melt[df_melt.Age=="1to4"].copy()
df_melt_5to14 = df_melt[df_melt.Age=="5to14"].copy()
df_melt_15to24 = df_melt[df_melt.Age=="15to24"].copy()
df_melt_25to34 = df_melt[df_melt.Age=="25to34"].copy()
df_melt_35to44 = df_melt[df_melt.Age=="35to44"].copy()
df_melt_45to54 = df_melt[df_melt.Age=="45to54"].copy()
df_melt_55to64 = df_melt[df_melt.Age=="55to64"].copy()
df_melt_65to74 = df_melt[df_melt.Age=="65to74"].copy()
df_melt_75to84 = df_melt[df_melt.Age=="75to84"].copy()
df_melt_85up = df_melt[df_melt.Age=="85up"].copy()

In [9]:
df_melt_1to4["CaseByDay"] = df_melt_1to4["Cases"].diff()
df_melt_1to4["HospitalCaseByDay"] = df_melt_1to4["HospitalisedCases"].diff()

df_melt_5to14["CaseByDay"] = df_melt_5to14["Cases"].diff()
df_melt_5to14["HospitalCaseByDay"] = df_melt_5to14["HospitalisedCases"].diff()

df_melt_15to24["CaseByDay"] = df_melt_15to24["Cases"].diff()
df_melt_15to24["HospitalCaseByDay"] = df_melt_15to24["HospitalisedCases"].diff()

df_melt_25to34["CaseByDay"] = df_melt_25to34["Cases"].diff()
df_melt_25to34["HospitalCaseByDay"] = df_melt_25to34["HospitalisedCases"].diff()

df_melt_35to44["CaseByDay"] = df_melt_35to44["Cases"].diff()
df_melt_35to44["HospitalCaseByDay"] = df_melt_35to44["HospitalisedCases"].diff()

df_melt_45to54["CaseByDay"] = df_melt_45to54["Cases"].diff()
df_melt_45to54["HospitalCaseByDay"] = df_melt_45to54["HospitalisedCases"].diff()

df_melt_55to64["CaseByDay"] = df_melt_55to64["Cases"].diff()
df_melt_55to64["HospitalCaseByDay"] = df_melt_55to64["HospitalisedCases"].diff()

df_melt_65to74["CaseByDay"] = df_melt_65to74["Cases"].diff()
df_melt_65to74["HospitalCaseByDay"] = df_melt_65to74["HospitalisedCases"].diff()

df_melt_75to84["CaseByDay"] = df_melt_75to84["Cases"].diff()
df_melt_75to84["HospitalCaseByDay"] = df_melt_75to84["HospitalisedCases"].diff()

df_melt_85up["CaseByDay"] = df_melt_85up["Cases"].diff()
df_melt_85up["HospitalCaseByDay"] = df_melt_85up["HospitalisedCases"].diff()

#### Step 7. Merge all above datasets again

In [10]:
df_Covid = pd.concat([df_melt_1to4,df_melt_5to14,df_melt_15to24,df_melt_25to34,df_melt_35to44,df_melt_45to54,df_melt_55to64,df_melt_65to74,df_melt_75to84,df_melt_85up],ignore_index=True, sort=False)

#### Step 8. Handling NaN values

In [11]:
df_Covid.loc[df_Covid["StatisticsProfileDate"]== start_date,"CaseByDay"] = df_Covid.loc[df_Covid["StatisticsProfileDate"]== start_date,"Cases"]

df_Covid.loc[df_Covid["StatisticsProfileDate"]== start_date,"HospitalCaseByDay"] = df_Covid.loc[df_Covid["StatisticsProfileDate"]== start_date,"HospitalisedCases"]

#### Step 9. Export to csv file

In [12]:
df_Covid.to_csv("Covid_Tan.csv", index=False)